In [1]:
import torch
import torchvision
import torch.nn
import numpy as np
import random
import os
import time
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
# 固定随机数种子，保证模型的可复现性
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    
set_seed(666)

In [3]:
#超参数
epochs = 6
batch_size = 64

In [4]:
# 预处理
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5,),(0.5,))]     )

# 加载数据集
train_data = torchvision.datasets.MNIST(root = './data', train = True, download = True, transform = transform)    
test_data = torchvision.datasets.MNIST(root = './data', train = False, download = True, transform = transform)      

train_dl = DataLoader(train_data, batch_size = batch_size)
test_dl = DataLoader(test_data, batch_size = batch_size)

print('len(train_loader)={}'.format(len(train_dl)))
print('len(train_loader)={}'.format(len(test_dl)))

len(train_loader)=938
len(train_loader)=157


In [5]:
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__() 
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, 5, 1, 2), 
            torch.nn.ReLU(),            
            torch.nn.MaxPool2d(2, 2))

        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, 5, 1, 0), 
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2))

        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(16 * 5 * 5, 120), 
            torch.nn.ReLU())

        self.fc2 = torch.nn.Sequential(
            torch.nn.Linear(120, 84),
            torch.nn.ReLU(),
            torch.nn.Linear(84, 10))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = LeNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(model)

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [6]:
# 训练
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_dl):
        images = Variable(images)
        labels = Variable(labels)
        optimizer.zero_grad() #梯度归零
        preds = model(images) #前向运算
        loss = criterion(preds, labels) #损失函数
        loss.backward() #反向传播
        optimizer.step() #通过梯度做进一步的参数更新
 
        if i % 200 ==0:
            print('epoch={}, batch={}, loss={:g}'.format(epoch, i, loss.item()))

epoch=0, batch=0, loss=2.31121
epoch=0, batch=200, loss=0.179477
epoch=0, batch=400, loss=0.0686001
epoch=0, batch=600, loss=0.13855
epoch=0, batch=800, loss=0.290489
epoch=1, batch=0, loss=0.0388803
epoch=1, batch=200, loss=0.0551373
epoch=1, batch=400, loss=0.0244812
epoch=1, batch=600, loss=0.0778633
epoch=1, batch=800, loss=0.185291
epoch=2, batch=0, loss=0.0500835
epoch=2, batch=200, loss=0.0320324
epoch=2, batch=400, loss=0.0114327
epoch=2, batch=600, loss=0.0614011
epoch=2, batch=800, loss=0.179756
epoch=3, batch=0, loss=0.0655621
epoch=3, batch=200, loss=0.057998
epoch=3, batch=400, loss=0.00500868
epoch=3, batch=600, loss=0.0421809
epoch=3, batch=800, loss=0.134482
epoch=4, batch=0, loss=0.00999517
epoch=4, batch=200, loss=0.0608411
epoch=4, batch=400, loss=0.00481056
epoch=4, batch=600, loss=0.0324359
epoch=4, batch=800, loss=0.098432
epoch=5, batch=0, loss=0.0102997
epoch=5, batch=200, loss=0.0210993
epoch=5, batch=400, loss=0.0270665
epoch=5, batch=600, loss=0.031082
epoch=

In [7]:
#测试准确性
total = 0
correct = 0

for test_image, test_label in test_dl:
    test_pred = model(test_image)
    pred = torch.argmax(test_pred, 1)
    total += test_label.size(0)
    correct += (pred == test_label).sum().item()

#打印准确率
print(correct / total)

0.9878
